In [ ]:
import networkx as nx
import re
import xlwt
import xlrd
from tqdm import tqdm
import numpy as np
import os
from tqdm import tqdm

Cat=xlrd.open_workbook('raw.xlsx')
cat = Cat.sheet_by_name('Sheet1')
num_edges_raw=len(cat.col_values(0))
#%%
def simplify(Graph):
    all_nodes = Graph.nodes
    simplified_nodes = []
    for node in all_nodes:
        in_degree = Graph.in_degree(node)
        out_degree = Graph.out_degree(node)
        if out_degree == 0:
            continue
        else:
            simplified_nodes.append(node)
    G_simplified = Graph.subgraph(simplified_nodes)
    print(len(simplified_nodes))
    return G_simplified
nodes = []
edges = []
for i in tqdm(range(num_edges_raw)):
    start_node = cat.col_values(0)[i]
    end_node = cat.col_values(1)[i]
    edge_attr = cat.col_values(2)[i]
    
    if start_node not in nodes:
        nodes.append(start_node)
    if end_node not in nodes:
        nodes.append(end_node)    
    
    if edge_attr == 'dual' or edge_attr == 'unknown':
        continue
    elif edge_attr == 'activator':
        edge_attr = 1
    elif edge_attr == 'repressor':
        edge_attr = -1
   
    edge = (start_node,end_node,edge_attr)
    if edge not in edges:
        edges.append(edge)

print(len(nodes),len(edges))
np.save('nodes.npy',np.array(nodes))

#%%

G = nx.DiGraph()
G.add_nodes_from(nodes)
G.add_weighted_edges_from(edges)
#%%
#sizes of strongly connected components
sccs = sorted(nx.strongly_connected_components(G),key=len, reverse=True)
wccs = sorted(nx.weakly_connected_components(G),key=len, reverse=True)
len_sccs = [len(c) for c in sorted(nx.strongly_connected_components(G),key=len, reverse=True)]    
len_wccs = [len(c) for c in sorted(nx.weakly_connected_components(G),key=len, reverse=True)] 

#%%
N = G.number_of_nodes()
#shortest_path_lengths = np.zeros((num_nodes,num_nodes))
nums_reached = []
nodes_reached = []

for j in tqdm(range(N)):
    num_reached = N
    nodes_reached_by_j = list(np.array(nodes))
    
    for k in range(N):
        #print(k)
        try:
            length=nx.shortest_path_length(G,source=nodes[j],target=nodes[k])
           
        except nx.NetworkXNoPath:
            #length = 10*N
            num_reached -= 1
            nodes_reached_by_j.remove(nodes[k])
    nums_reached.append(num_reached)
    nodes_reached.append(nodes_reached_by_j)

idx_reached_most = nums_reached.index(max(nums_reached))          
reached_most = nodes_reached[idx_reached_most]
reached_most.append(nodes[idx_reached_most])

print(len(reached_most))
def simplify(Graph):
    all_nodes = Graph.nodes
    simplified_nodes = []
    for node in all_nodes:
        in_degree = Graph.in_degree(node)
        out_degree = Graph.out_degree(node)
        if out_degree == 0:
            continue
        else:
            simplified_nodes.append(node)
    G_simplified = Graph.subgraph(simplified_nodes)
    print(len(simplified_nodes))
    return G_simplified

    
G_wcc = G.subgraph(list(wccs[0]))

l = [e for e in G_wcc.in_edges('cpxR',data=True) if e[2]['weight']>0]

G_reach = G.subgraph(reached_most)
G_ws2 = simplify(simplify(simplify(G_wcc)))
G_rs2 = simplify(simplify(simplify(G_reach)))

nx.write_gml('rs2.gml',G_rs2)